In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../..")

# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units
from nff.io import NeuralFF, AtomsBatch

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [3]:
dataset = Dataset.from_file('data/dataset.pth.tar')


### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [4]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props
                   )


### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [5]:
nff_ase = NeuralFF.from_file('sandbox/', device=0)

Assigning this calculator to `atoms` is easy:

In [6]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `thermostat_params`: keyword arguments for ase.Integrator class, will be different case-by-case
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [7]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
#     'thermostat': NoseHoover,   # or Langevin or NPT or NVT or Thermodynamic Integration
#     'thermostat_params': {'timestep': 0.5 * units.fs, "temperature": 120.0 * units.kB,  "ttime": 20.0}
    'thermostat': VelocityVerlet,  
    'thermostat_params': {'timestep': 0.5 * units.fs},
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 3,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [8]:
nve = Dynamics(atoms, md_params)
nve.run()

/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/ase/atoms.py:968: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)


Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.1497      -0.2563       0.4061   349.0

0.0050           0.1495      -0.2296       0.3791   325.9

0.0100           0.1483      -0.3977       0.5460   469.3

0.0150           0.1481      -0.2730       0.4211   362.0

0.0200           0.1474      -0.3391       0.4865   418.2

0.0250           0.1483      -0.4296       0.5779   496.8

0.0300           0.1494      -0.2021       0.3515   302.2

0.0350           0.1493      -0.2785       0.4278   367.7

0.0400           0.1495      -0.2821       0.4316   371.0

0.0450           0.1494      -0.1838       0.3332   286.4

0.0500           0.1485      -0.3953       0.5437   467.4

0.0550           0.1495      -0.1611       0.3106   267.0

0.0600           0.1502      -0.0800       0.2302   197.9

0.0650           0.1504      -0.1538       0.3042   261.5

0.0700           0.1506      -0.1133       0.2639   226.8

0.0750           0.1491      -0.2431       0.3922   337.2


### Models with directed neighbor lists

The default assumes that you're using SchNet, which uses an undirected neighbor list to save memory. If you're using Painn, DimeNet, or any model with directional information, you will need a directed neighbor list. If you don't specify this then you will get an error. 

While these models automatically make any neighbor list into a directed one when called, the same is not true of periodic boundary conditions. The dataest offsets from the PBC depend on whether the dataset neighbor list is directed or not.

In [9]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
Painn needs a directed neighbor list


If you do then you will be fine!

In [10]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   directed=True)
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)

try:
    nve.run()
except Exception as e:
    print(e)
    import pdb
    pdb.post_mortem()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.5324      -0.1617       0.6941   596.6

0.0050           0.5332      -0.0530       0.5861   503.8

0.0100           0.5320      -0.0389       0.5710   490.8

0.0150           0.5365       0.0705       0.4660   400.6

0.0200           0.5394       0.0145       0.5248   451.1

0.0250           0.5383       0.2448       0.2934   252.2

0.0300           0.5336      -0.0760       0.6096   524.0

0.0350           0.5341      -0.0614       0.5955   511.9

0.0400           0.5311      -0.0386       0.5698   489.8

0.0450           0.5302      -0.1988       0.7291   626.7

0.0500           0.5334      -0.0635       0.5968   513.0

0.0550           0.5345      -0.0539       0.5884   505.7

0.0600           0.5346      -0.0884       0.6230   535.5

0.0650           0.5321      -0.0729       0.6049   520.0

0.0700           0.5351      -0.0080       0.5431   466.8

0.0750           0.5318      -0.1914       0.7232   621.7


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

# Additional properties and options

You can also specify the properties that you want the calculator to predict. The default is `energy` and `forces`, but you can also add `stress`.

However, if you run this for ethanol you will get an error, because it doesn't have any lattice vectors:

In [11]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]      ---------------------- stress [GPa] -----------------------
You have 0 lattice vectors: volume not defined


If we make an artificial unit cell, it all works fine:

In [12]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)
atoms.set_cell(2 * np.identity(3))
nve = Dynamics(atoms, md_params)
nve.run()



Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]      ---------------------- stress [GPa] -----------------------
0.0000           0.4264      -0.0737       0.5001   429.9    -99.151    -24.362     31.056      1.412      7.989    -56.938

0.0050           0.4264      -0.2205       0.6469   556.1     43.490     61.182     -0.875     -9.883      2.269     30.666

0.0100           0.4244      -0.0214       0.4458   383.2     30.041     59.364     -0.351    -20.328    -21.915     68.927

0.0150           0.4249      -0.1070       0.5319   457.2    101.898     39.245    -17.539    -18.676    -16.221     23.003

0.0200           0.4263      -0.1522       0.5785   497.3     54.157    -36.781     22.149      8.034     19.317     38.156

0.0250           0.4277       0.0726       0.3550   305.2     -1.747    -43.600      0.491     29.832     20.963    -92.663

0.0300           0.4273      -0.0954       0.5227   449.3    -77.737      1.212     10.841     -0.684    -15.015     -3.378

0.0

Since `stress` is an implemented property, we can get the stress on the atoms:

In [13]:
atoms.get_stress()

array([ 0.6590634 ,  0.30095565,  0.10327584, -0.03814176,  0.00766306,
       -0.2284697 ])

You can additionally request larger offsets for periodic structures than the default. To do so, use `requires_large_offsets=True` when making the atoms:

In [14]:
lattice = [[1.155363155, 0.0, 0.0], 
           [0.0, 1.155363155, 0.0], 
           [0.0, 0.0, 1.07085842]]

# default of `requires_large_offsets` is False
old_atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                       numbers=props['nxyz'][:, 0], 
                       props=props,
                       cell=lattice,
                       pbc=True
                       )

new_atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                       numbers=props['nxyz'][:, 0], 
                       props=props,
                       requires_large_offsets=True,
                       cell=lattice,
                       pbc=True
                       )

If we compare the offsets in the new and old `atoms` objects, we see that there are differences:

In [15]:
old_nbrs, old_offsets = old_atoms.update_nbr_list()
new_nbrs, new_offsets = new_atoms.update_nbr_list()

print((old_offsets.to_dense() == new_offsets.to_dense()).all().item())
print(old_offsets)
print(new_offsets)


False
tensor(indices=tensor([[ 0,  0,  1,  1,  2,  2,  3,  3,  4,  5,  5,  5,  6,  6,
                         6,  7,  8,  9,  9,  9, 10, 10, 10, 11, 11, 12, 12, 13,
                        14, 15, 15, 15, 16, 16, 16, 17, 17, 18, 18, 18, 19, 19,
                        20, 21, 22, 22, 22, 23, 23, 24, 24, 24, 25, 25, 25, 26,
                        26, 26, 27, 27, 27, 28, 28, 29, 29, 29, 30, 30, 30, 31,
                        31, 31, 32, 33, 34, 34, 34, 35, 35, 35],
                       [ 0,  1,  0,  1,  1,  2,  1,  2,  0,  0,  1,  2,  0,  1,
                         2,  0,  0,  0,  1,  2,  0,  1,  2,  0,  1,  1,  2,  2,
                         0,  0,  1,  2,  0,  1,  2,  0,  1,  0,  1,  2,  0,  2,
                         0,  2,  0,  1,  2,  0,  1,  0,  1,  2,  0,  1,  2,  0,
                         1,  2,  0,  1,  2,  0,  1,  0,  1,  2,  0,  1,  2,  0,
                         1,  2,  0,  2,  0,  1,  2,  0,  1,  2]]),
       values=tensor([ 1.1554,  1.1554, -1.1554,  1.1554, -1.1

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [16]:
import nglview as nv
from ase.io import Trajectory

Displaying the trajectory:

In [17]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

NGLWidget(max_frame=19)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!